## Read GRN from  google sheet

In [18]:
# import library
import pandas as pd
import numpy as np
import requests
import io

try:
    from StringIO import StringIO
except ImportError:
    
    from io import StringIO
    from io import BytesIO
from decimal import Decimal
from datetime import datetime
# google_sheet_url = 'https://docs.google.com/spreadsheets/d/1Y0CylvsLpJxVRNa1Ha5JBBLmVfL7TS7x4gZuNS2tylw/export?format=xlsx'
google_sheet_url = 'https://docs.google.com/spreadsheets/d/1iTCEuO-qc5yH4uByxtHJiWzs_zztDdFg/export?format=xlsx'
r = requests.get(google_sheet_url)
data = r.content
df = pd.read_excel(BytesIO(data),sheet_name='11 April MISS').fillna('0')
df.head()
# print(df.dtypes)

df.new_quantity = df.new_quantity.astype(int)
df.price_per_unit = df.price_per_unit.astype(int)
df.price = df.price.astype(int)
df.total_price = df.total_price.astype(int)
df.igst_value = df.igst_value.astype(int)
df.cgst_value = df.cgst_value.astype(int)
df.sgst_value = df.sgst_value.astype(int)
df.expiry_date = df.expiry_date.astype(int)
# print(df.dtypes)
df


,grn_number,grn_date,bill_number,bill_date,purchase_company_name,po_number,po_date,dc_number,dc_date,by_product_code,new_quantity,price_per_unit,price,total_price,igst_value,cgst_value,sgst_value,expiry_date
0,GRN2200022,2022-04-11,2008,2022-04-11,NDC,0,0,0,0,KK01,147,0,0,0,0,0,0,0


In [19]:
grn_numbers = list(set(list(df['grn_number'])))
for grn in grn_numbers:
    print(df['by_product_code'])
    first_df = df[df['grn_number'] == grn].head(1).to_dict('r')[0]
    if PurchaseCompany.objects.filter(company_name=first_df['purchase_company_name']).exists():
        purchase_company_id = PurchaseCompany.objects.get(company_name=first_df['purchase_company_name']).id
    else:
        purchase_company_obj = PurchaseCompany.objects.create(company_name=first_df['purchase_company_name'],
                                                             created_by_id=1)
        purchase_company_obj.save()
    goods_receipt_mater = GoodsReceiptMaster(grn_number=grn,
                                    grn_date=first_df['grn_date'],
                                    bill_number=first_df['bill_number'],
                                    bill_date=first_df['bill_date'],
                                    purchase_company_id=purchase_company_id,
                                    created_by_id=1,
                                    modified_by_id=1,
                                    )
    if first_df['po_number'] != '0':
        goods_receipt_mater.po_number = first_df['po_number']
    if first_df['po_date'] != '0':
        goods_receipt_mater.po_date = first_df['po_date']
    if first_df['dc_number'] != '0':
        goods_receipt_mater.dc_number = first_df['dc_number']
    if first_df['dc_date'] != '0':
        goods_receipt_mater.dc_date = first_df['dc_date']
    goods_receipt_mater.save()
    for index, row in df[df['grn_number'] == grn].iterrows():
        print(row['by_product_code'])
        by_product = ByProduct.objects.get(code=row['by_product_code'])
        goods_receipt_record_obj = GoodsReceiptRecord(goods_receipt_master_id=goods_receipt_mater.id,
                                                      by_product_id=by_product.id,
                                                      quantity_at_receipt=row['new_quantity'],
                                                      quantity_now=row['new_quantity'],
                                                      quantity_now_time=row['grn_date'],
                                                      price_per_unit=row['price_per_unit'],
                                                      price=row['price'],
                                                      total_price=row['total_price'],
                                                      goods_receipt_code='from old excel',
                                                      created_by_id=1,
                                                      modified_by_id=1,
                                                      )
        if row['igst_value'] != 0:
            goods_receipt_record_obj.igst_value = row['igst_value']
        if row['cgst_value'] != 0:
            goods_receipt_record_obj.cgst_value = row['cgst_value']
        if row['sgst_value'] != 0:
            goods_receipt_record_obj.sgst_value = row['sgst_value']
        if row['expiry_date'] != 0:
            goods_receipt_record_obj.expiry_date = row['expiry_date']
        goods_receipt_record_obj.save()
        daily_goods_receipt_record_obj = GoodsReceiptRecordForDaily(sale_date=row['grn_date'],
                                                      by_product_id=by_product.id,
                                                      quantity_at_receipt=row['new_quantity'],
                                                      quantity_now=row['new_quantity'],
                                                      quantity_now_time=row['grn_date'],
                                                      price_per_unit=row['price_per_unit'],
                                                      total_price=row['total_price'],
                                                      grn_number=row['grn_number'],
                                                      )
        daily_goods_receipt_record_obj.save()

        # add data to Main Stock
        if ByProductCurrentAvailablity.objects.filter(by_product_id=by_product.id).exists():
            main_input_stock_obj = ByProductCurrentAvailablity.objects.get(by_product_id=by_product.id)
            main_input_stock_obj.quantity_now = main_input_stock_obj.quantity_now + row['new_quantity']
            main_input_stock_obj.quantity_now_time = row['grn_date']
            main_input_stock_obj.modified_by_id = 1
            main_input_stock_obj.save()
        else:
            main_input_stock_obj = ByProductCurrentAvailablity(by_product_id=by_product.id,
                                                quantity_now=row['new_quantity'],
                                                quantity_now_time=row['grn_date'],
                                                created_by_id=1,
                                                modified_by_id=1)
            main_input_stock_obj.save()
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        

0    KK01
Name: by_product_code, dtype: object
KK01


<ipython-input-19-59692c20ec98>:4: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  first_df = df[df['grn_number'] == grn].head(1).to_dict('r')[0]
/home/steve/Software/virtualenv/aavin/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField GoodsReceiptMaster.grn_date received a naive datetime (2022-04-11 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/steve/Software/virtualenv/aavin/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField GoodsReceiptMaster.bill_date received a naive datetime (2022-04-11 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/steve/Software/virtualenv/aavin/lib/python3.9/site-